# E91 Protocol - SOLUTION

**Complete solution for instructors**

In [2]:
import random
import numpy as np
from qiskit import QuantumCircuit
import sys
sys.path.append('utils')
import encryption_algorithms as enc

# Import solved CHSH functions from the utils module
from utils.chsh_core import *

print(" Setup complete!")
print("\n Imported CHSH functions from utils/chsh_core.py:")
print("   • run_circuit")
print("   • create_bell_pair_singlet_state") 
print("   • apply_basis_transformation")
print("   • measure_bell_pair")
print("   • create_eavesdropped_state")
print("   • organize_measurements_by_basis")
print("   • calculate_correlations")
print("   • calculate_chsh_value")

# Set seed for reproducibility  
GLOBAL_SEED = 91
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)

# E91 Protocol Constants
ALICE_BASES = ['0', '45', '90']      # Alice's available bases
BOB_BASES = ['45', '90', '135']      # Bob's available bases
ALICE_CHSH_BASES = ['0', '90']
BOB_CHSH_BASES = ['45', '135']
CHSH_BASIS_PAIRS = [(a, b) for a in ALICE_CHSH_BASES for b in BOB_CHSH_BASES]

# Interpretation:
#   |S| < 2.0       → Classical regime (no violation)
#   2.0 < |S| < 2.5 → Caution: theoretical violation, but may be due to noise or
#                     insufficient statistics (this boundary is a pedagogical choice,
#                     not a scientifically established threshold)
#   |S| > 2.5       → Robust violation, clearly quantum
BELL_INEQUALITY_THRESHOLD = 2.5

EVE_PERCENTAGE_COMPROMISED = 0.7 # Eve intercepts 70% of the pairs

print(f"\nAlice's bases: {ALICE_BASES}")
print(f"Bob's bases: {BOB_BASES}")
print(f"Key generation pairs: (45,45) and (90,90)")
print(f"CHSH test basis pairs : {CHSH_BASIS_PAIRS}")

 Setup complete!

 Imported CHSH functions from utils/chsh_core.py:
   • run_circuit
   • create_bell_pair_singlet_state
   • apply_basis_transformation
   • measure_bell_pair
   • create_eavesdropped_state
   • organize_measurements_by_basis
   • calculate_correlations
   • calculate_chsh_value

Alice's bases: ['0', '45', '90']
Bob's bases: ['45', '90', '135']
Key generation pairs: (45,45) and (90,90)
CHSH test basis pairs : [('0', '45'), ('0', '135'), ('90', '45'), ('90', '135')]


## 1) Generate Random Bases

Alice and Bob each randomly choose measurement bases for each qubit.
Why: independent, random choices prevent Eve from predicting measurement settings.

In [3]:
def generate_random_bases(length: int, options: list) -> list:
    """
    Generate a list of random bases.
    
    Args:
        length: Number of bases to generate
        options: List of possible bases (e.g., ['0', '45', '90'])
    
    Returns:
        List of randomly chosen bases
    """
    # Simply choose 'length' random bases from the options
    return [random.choice(options) for _ in range(length)]


In [5]:
# Test
num_random_basis = 10
test_bases = generate_random_bases(num_random_basis, ['0', '45', '90'])
print(f"Generated bases: {test_bases}")
print(f"Expected: {num_random_basis} random values from ['0', '45', '90']")

Generated bases: ['45', '45', '45', '0', '0', '90', '45', '45', '0', '0']
Expected: 10 random values from ['0', '45', '90']


## 2) Create Bell Pairs

Why: E91 starts with many identical entangled pairs (same idea as CHSH).


In [6]:
def create_list_bell_pairs(num_pairs: int) -> list:
    """
    Create a list of Bell pair circuits.
    
    Args:
        num_pairs: Number of Bell pairs to create
    
    Returns:
        List of QuantumCircuit objects (Bell pairs)
    """
    # Create 'num_pairs' identical Bell states using the function from NB1
    return [create_bell_pair_singlet_state() for _ in range(num_pairs)]



## 3) Measure All Pairs

In [7]:
def measure_all_pairs(bell_pairs: list, alice_bases: list, bob_bases: list) -> list:
    """
    Measure each Bell pair with the corresponding bases.
    
    Returns:
        List of measurement results ('00', '01', '10', '11')
    """
    results = []
    
    # Loop through every pair and measure it using the chosen bases
    for qc, a_base, b_base in zip(bell_pairs, alice_bases, bob_bases):
        # measure_bell_pair is the function from Notebook 1
        result = measure_bell_pair(qc, a_base, b_base)
        results.append(result)
    
    return results


## 4) Sift Results (Key Generation vs Bell Test)

**Sifting rules:**
- Same basis (45,45) or (90,90) → **Key generation**
- CHSH bases → **Bell test**
- Other combinations → **Discard**

**This is the heart of E91** — it decides which data becomes key vs security test.

Exercise: Implement `extract_e91_key_and_bell_test_data()`

### Sifting intuition (why it works)

- **Same basis** → strong (anti-)correlation → safe for key bits.
- **Different bases** → more random → useful for Bell test or discarded.

This mirrors what happens with polarizers in the lab experiment.

---

In [8]:
# 4: Sift results
def extract_e91_key_and_bell_test_data(results, alice_bases, bob_bases):
    key_results = [] # keep output results for same bases
    chsh_results = [] # keep output results for bases combinations that belong to CHSH test
    chsh_alice_bases = [] # keep alice bases that are in (combinations that belong to CHSH test)
    chsh_bob_bases = [] # same as alice
    # note : at index i, we have chsh output related to alice and bob base
    
    for i, (result, a_base, b_base) in enumerate(zip(results, alice_bases, bob_bases)):
        if a_base == b_base:
            key_results.append(result)
        elif (a_base, b_base) in CHSH_BASIS_PAIRS:
            chsh_results.append(result)
            chsh_alice_bases.append(a_base)
            chsh_bob_bases.append(b_base)
    
    return {
        'key_results': key_results,
        'chsh_results': chsh_results,
        'chsh_alice_bases': chsh_alice_bases,
        'chsh_bob_bases': chsh_bob_bases,
    }

## 5) Check for Eavesdropping

Use the Bell test data to verify security.
If $|S| > 2$, entanglement is intact and the channel is secure.

*(This reuses the CHSH logic from Notebook 1. Provided for speed.)*


In [9]:
def check_for_eavesdropping(chsh_results: list, chsh_alice_bases: list, chsh_bob_bases: list) -> dict:
    """
    Run CHSH test to check for eavesdropping.
    """
    # 1. Organize the raw measurement results by basis pair
    bell_results = organize_measurements_by_basis(chsh_results, chsh_alice_bases, chsh_bob_bases)
    
    # 2. Calculate the correlation for each basis pair
    correlations = calculate_correlations(bell_results)
    
    # 3. Compute the CHSH 'S' value
    chsh_value = calculate_chsh_value(correlations)
    
    # 4. Check against the threshold (2.5)
    is_secure = chsh_value > BELL_INEQUALITY_THRESHOLD
    
    return {
        'chsh_value': chsh_value,
        'is_secure': is_secure
    }


## 6) Complete E91 Protocol

Put it all together!
Orchestration shows the full pipeline from entanglement to a shared key.

Exercise: Complete `run_e91_protocol()`

In [10]:
EVE_PERCENTAGE_COMPROMISED = 0.7

def run_e91_protocol(num_pairs: int = 2000, eavesdropping: bool = False) -> str:
    """
    Run the complete E91 QKD protocol.
    
    Args:
        num_pairs: Number of Bell pairs to generate
        eavesdropping: If True, simulate Eve intercepting pairs
    
    Returns:
        Shared secret key (string of 0s and 1s) or None if insecure
    """
    print("="*60)
    print("E91 QUANTUM KEY DISTRIBUTION PROTOCOL")
    print("="*60)
    
    # Step 1: Generate Bell pairs
    print(f"\n1. Generating {num_pairs} Bell pairs...")
    bell_pairs = create_list_bell_pairs(num_pairs)
    
    # Step 2: Simulate eavesdropping (optional)
    if eavesdropping:
        compromised = int(num_pairs * EVE_PERCENTAGE_COMPROMISED)
        print(f"\n EVE intercepts {compromised} pairs ({EVE_PERCENTAGE_COMPROMISED*100}%)")
        bell_pairs = [create_eavesdropped_state(qc) for qc in bell_pairs[:compromised]] + bell_pairs[compromised:]
    
    # Step 3: Generate random bases
    print("\n2. Alice and Bob choose random bases...")
    alice_bases = generate_random_bases(num_pairs, ALICE_BASES)
    bob_bases = generate_random_bases(num_pairs, BOB_BASES)
    
    # Step 4: Measure all pairs
    print("\n3. Measuring all pairs...")
    results = measure_all_pairs(bell_pairs, alice_bases, bob_bases)
    
    # Step 5: Sift results
    print("\n4. Sifting results...")
    data = extract_e91_key_and_bell_test_data(results, alice_bases, bob_bases)
    
    print(f"   Key generation pairs: {len(data['key_results'])}")
    print(f"   Bell test pairs: {len(data['chsh_results'])}")
    
    # Step 6: Security check
    print("\n5. Running CHSH security test...")
    security = check_for_eavesdropping(
        data['chsh_results'],
        data['chsh_alice_bases'],
        data['chsh_bob_bases']
    )
    
    print(f"\n   CHSH Value: {security['chsh_value']:.4f}")
    print(f"   Classical limit: 2.0")
    
    if not security['is_secure']:
        print("\n SECURITY CHECK FAILED!")
        print("   Possible eavesdropping detected.")
        print("   Key generation ABORTED.")
        return None
    
    print("\n SECURITY CHECK PASSED!")
    
    # Step 7: Extract key
    print("\n6. Extracting shared key...")
    
    # Alice's bits (first bit of each result)
    alice_key = ''.join([r[0] for r in data['key_results']])
    
    # Bob's bits (second bit of each result)
    # For singlet state: anti-correlated, so Bob flips his bits
    bob_key = ''.join(['1' if r[1] == '0' else '0' for r in data['key_results']])
    
    # Verify keys match
    if alice_key != bob_key:
        mismatches = sum(a != b for a, b in zip(alice_key, bob_key))
        print(f"\n Warning: {mismatches} bit mismatches (noise or Eve)")
    else:
        print("\n Keys match perfectly!")
    
    print(f"\n   Key length: {len(alice_key)} bits")
    print(f"   Key (first 50 bits: {alice_key[:50]}...")
    
    return alice_key


##  Run E91 Protocol (No Eavesdropping)

In [11]:
# Run without Eve
key = run_e91_protocol(num_pairs=2000, eavesdropping=False)

E91 QUANTUM KEY DISTRIBUTION PROTOCOL

1. Generating 2000 Bell pairs...

2. Alice and Bob choose random bases...

3. Measuring all pairs...

4. Sifting results...
   Key generation pairs: 444
   Bell test pairs: 901

5. Running CHSH security test...
Correlations:
  E('90', '135') = -0.7021
  E('90', '45') = -0.7227
  E('0', '135') = 0.6456
  E('0', '45') = -0.7277

   CHSH Value: 2.7981
   Classical limit: 2.0

 SECURITY CHECK PASSED!

6. Extracting shared key...

 Keys match perfectly!

   Key length: 444 bits
   Key (first 50 bits: 11101011000100000011100100100010101110100010001110...


##  Run E91 Protocol (With Eavesdropping)

In [12]:
# Reset and run with Eve
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)
MANUAL_SIMULATOR_SEED_COUNTER = GLOBAL_SEED
# num_pairs=800 juts to show the concep. 
key_with_eve = run_e91_protocol(num_pairs=800, eavesdropping=True)

E91 QUANTUM KEY DISTRIBUTION PROTOCOL

1. Generating 800 Bell pairs...

 EVE intercepts 560 pairs (70.0%)

2. Alice and Bob choose random bases...

3. Measuring all pairs...

4. Sifting results...
   Key generation pairs: 176
   Bell test pairs: 350

5. Running CHSH security test...
Correlations:
  E('90', '135') = -0.2079
  E('90', '45') = -0.1111
  E('0', '135') = 0.6842
  E('0', '45') = -0.7108

   CHSH Value: 1.7141
   Classical limit: 2.0

 SECURITY CHECK FAILED!
   Possible eavesdropping detected.
   Key generation ABORTED.


## Encrypt and Decrypt Messages

In [12]:
# Decrypt messages
random.seed(GLOBAL_SEED)
np.random.seed(GLOBAL_SEED)
MANUAL_SIMULATOR_SEED_COUNTER = GLOBAL_SEED

key = run_e91_protocol(num_pairs=2000, eavesdropping=False)

print(f"Key generated: {key[:50]}... (length: {len(key)} bits)")

E91 QUANTUM KEY DISTRIBUTION PROTOCOL

1. Generating 2000 Bell pairs...

2. Alice and Bob choose random bases...

3. Measuring all pairs...

4. Sifting results...
   Key generation pairs: 441
   Bell test pairs: 898

5. Running CHSH security test...
CHSH test pairs: 898
Correlations:
  E('0', '135') = 0.7447
  E('0', '45') = -0.6042
  E('90', '135') = -0.7191
  E('90', '45') = -0.7034

   CHSH Value: 2.7714

 SECURITY CHECK PASSED!

6. Extracting shared key...

 Keys match perfectly!

   Key length: 441 bits
   Key (first 50 bits): 11010101010100100011011000000111010001001100100001...
Key generated: 11010101010100100011011000000111010001001100100001... (length: 441 bits)


## Use the key to encrypt/decrypt

In [13]:
# use the generated key to encrypt and decrypt a message using XOR
# I use the key generated without Eve for encryption, as the one with Eve is likely compromised
message = '"What I cannot create, I do not understand." - Richard Feynman'
print(f"\nOriginal: {message}")

encrypted = enc.encrypt_xor_repeating_key(message, key)
print(f"\nEncrypted (hex): {encrypted[:50]}...")

decrypted = enc.decrypt_xor_repeating_key(encrypted, key)
print(f"\nDecrypted: {decrypted}")


Original: "What I cannot create, I do not understand." - Richard Feynman

Encrypted (hex): 136659514510781153515e5f5f44105342555045541c107810...

Decrypted: "What I cannot create, I do not understand." - Richard Feynman


## Challenge Messages to encrypt/decrypt

In [14]:
def encrypt_and_save_messages(dict_message: dict, key: str, filename: str = "encrypted_messages.txt"):
    """
    Encrypt all messages in dict_message using XOR with the given key and save to a file.

    Args:
        dict_message (dict): Dictionary of messages to encrypt.
        key (str): The key to use for XOR encryption.
        filename (str): The file to save encrypted messages to.
    """
    print("\nEncrypting all messages from dict_message:")
    list_encrypted = []
    for i, message in dict_message.items():
        encrypted = enc.encrypt_xor_repeating_key(message, key)
        list_encrypted.append(encrypted)
        print(f"Message {i}: {encrypted}")

    with open(filename, "w") as f:
        for i, encrypted in enumerate(list_encrypted, 1):
            f.write(f"Message {i}: {encrypted}\n")
    print(f"Encrypted messages saved in {filename}")


def decrypt_and_print_messages(key: str, filename: str = "encrypted_messages.txt"):
    """
    Read encrypted messages from a file, decrypt them using XOR with the given key, and print.

    Args:
        key (str): The key to use for XOR decryption.
        filename (str): The file to read encrypted messages from.
    """
    print("\nDecrypting all messages from", filename)
    with open(filename, "r") as f:
        for line in f:
            if ": " in line:
                msg_id, encrypted = line.split(": ", 1)
                decrypted = enc.decrypt_xor_repeating_key(encrypted.strip(), key)
                print(f"{msg_id}: {decrypted}")


In [15]:
# Messages to encrypt/decrypt
dict_message = {
   1: '"Nature isn\'t classical, dammit, and if you want to make a simulation of nature, you\'d better make it quantum mechanical.",  # Richard Feynman',
   2: '"If you think you understand quantum mechanics, you don\'t understand quantum mechanics.",  # Richard Feynman',
   3: '"Not only is the universe stranger than we think, it is stranger than we can think.",  # Werner Heisenberg',
   4: '"Anyone who is not shocked by quantum theory has not understood it.",  # Niels Bohr',
   5: '"Entanglement is not one but rather the characteristic trait of quantum mechanics.",  # Erwin Schrödinger',
   6: '"In quantum computing, information is physical and the laws of physics determine what can be computed.",  # David Deutsch',
   7: '"What I cannot create, I do not understand." Richard Feynman.',
   8: '"God does not play dice with the universe."  # Albert Einstein',
   9: '"Quantum mechanics is very impressive. But an inner voice tells me that it is not yet the real thing."  # Albert Einstein',
   10: '"Spooky action at a distance."  # Albert Einstein on quantum entanglement'
}

In [16]:
# encript and decrypt all messages from dict_message (using Xor only)
print("\nEncrypting and saving messages...")
encrypt_and_save_messages(dict_message, key)


Encrypting and saving messages...

Encrypting all messages from dict_message:
Message 1: 137f50444442541159435e164410535c5143425852515c1d1054505d5d59451c11515f5511595710495f441047515f4511445e115d505b5410511143585d445d5045585e5e105f57115e51454542541d10485f45175410535544445442105d515a54115945104144515e44455c105c555258515f5852515d1e121d1011121063595358514255117654495f5d505f
Message 2: 13785710485f44114458595f5b10495f4510445f5555424244515f54104144515f44445c115d545358515f5953431d11485f4411545e5e164410445e5555434245505f55104145505f44455c105d545258505e5953431e131c1010121062595359504354117655485e5d515e
Message 3: 137f5e44115f5f5d491059421044585510455f58475542425510424442515f575442114559515f104755114458595f5a1d10584510584311434443515f575443114559505e1047541153515f104459585e5a1e121c101012106755435e55421079545843545e52544257
Message 4: 13705f495e5e541147585f115943105e5f441142595f535a5554115249104045505e45445c104558555f434910585042115e5e4510445e55554242445e5f551158451f131c101012117e59545c4311735f

In [17]:
# For your challenge: must generate the correct key using the E91 protocol to decrypt the messages. Without the key, decryption is not feasible.
# change the path to the encrypted file as needed
path_to_encrypted_file = R"encrypted_messages.txt"
decrypt_and_print_messages(key, filename=path_to_encrypted_file)



Decrypting all messages from encrypted_messages.txt
Message 1: "Nature isn't classical, dammit, and if you want to make a simulation of nature, you'd better make it quantum mechanical.",  # Richard Feynman
Message 2: "If you think you understand quantum mechanics, you don't understand quantum mechanics.",  # Richard Feynman
Message 3: "Not only is the universe stranger than we think, it is stranger than we can think.",  # Werner Heisenberg
Message 4: "Anyone who is not shocked by quantum theory has not understood it.",  # Niels Bohr
Message 5: "Entanglement is not one but rather the characteristic trait of quantum mechanics.",  # Erwin Schrödinger
Message 6: "In quantum computing, information is physical and the laws of physics determine what can be computed.",  # David Deutsch
Message 7: "What I cannot create, I do not understand." Richard Feynman.
Message 8: "God does not play dice with the universe."  # Albert Einstein
Message 9: "Quantum mechanics is very impressive. But an inner 